In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_inventory_sloc_1000")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","""VCM_DWH_PRD.F_STAGING_INVENTORY_SLOC_1000, VCM_STG_INF.SAP_0PLANT"""
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000
(
  hash_id int,
  calday DATE,
  store_id STRING,
  product_id STRING,
  closing_stock_quantity FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_f_inventory_sloc_1000 AS
WITH table_date AS 
(
    SELECT col AS date_gen
    FROM EXPLODE(SEQUENCE(DATE {START_DATE}, DATE {END_DATE}, INTERVAL 1 DAY)) AS date_gen
    WHERE col <= CURRENT_DATE() 
)
SELECT 
* EXCEPT(rn, date_from, date_to) 
FROM (
    SELECT
        farm_fingerprint(CONCAT(
            IFNULL(CAST(date_gen AS STRING), ""), 
            IFNULL(CAST(store_id AS STRING), ""), 
            IFNULL(CAST(product_id AS STRING), "")
        )) AS hash_id,
        date_gen AS calday,
        store_id,
        product_id,
        closing_stock_quantity,
        ROW_NUMBER() OVER (PARTITION BY store_id, product_id, CAST(closing_stock_quantity AS BIGINT) ORDER BY date_gen ASC) AS rn,
        date_from,
        DATE_SUB(IFNULL(date_to, '2400-01-01'), 1) AS date_to
    FROM table_date 
    INNER JOIN 
    (
        SELECT
            calday,
            store_id,
            product_id,
            ROUND(SUM(closing_stock_quantity) OVER (PARTITION BY store_id, product_id ORDER BY calday ASC), 3) AS closing_stock_quantity,
            calday AS date_from,
            LEAD(calday) OVER (PARTITION BY store_id, product_id ORDER BY calday ASC) AS date_to
        FROM 
        (
            SELECT  
                calday, store_id, product_id, closing_stock_quantity 
            FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory_sloc_1000
            WHERE calday >= {START_DATE} AND calday <= {END_DATE} AND calday <= CURRENT_DATE()
            UNION ALL
            SELECT 
                calday, store_id, product_id, closing_stock_quantity 
            FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000 
            WHERE calday = DATE_SUB({START_DATE}, 1)
        ) a
    ) i
    ON date_gen >= calday AND (date_gen < date_to OR date_to IS NULL)
) a 
WHERE closing_stock_quantity != 0 OR rn = 1
""")


In [0]:
CALDAY_IN_STR = ''
calday_query1 = f"""
SELECT  
CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(CALDAY, 'yyyy-MM-dd') AS STRING))), "'") AS CALDAY_IN_STR 
,COALESCE(MIN(CALDAY),'1900-01-01') AS start_date
,COALESCE(MAX(CALDAY),'2400-01-01') AS end_date
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory_sloc_1000
WHERE calday > (SELECT COALESCE(MAX(calday),'1900-01-01') FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000) 
"""
calday_df1 = spark.sql(calday_query1)
CALDAY_IN_STR = calday_df1.collect()[0][0]
start_date = calday_df1.collect()[0][1]
end_date = calday_df1.collect()[0][2]
print(start_date, end_date,CALDAY_IN_STR)

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_{target_table}
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(
            f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000
        WHERE calday IN ({calday_in_str})
        """
        ).display()
    else:
        print("no valid calday's found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_2000 can't be deleted using calday's from temp_f_inventory_sloc_1000"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000 ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000 can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"